# Gradient-Based Training of WANNs

In [1]:
import torch
from torch.autograd import Variable
import numpy as np

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score, precision_score, recall_score

from model import Model

In [2]:
dataset = load_iris()
#train_X, test_X, train_y, test_y = train_test_split(dataset['data'],
#                                                    dataset['target'], test_size=0.2)


# Just going for fitting here. (Todo: Change stuff in paper for usefull comparison)
train_X = dataset['data']
train_y = dataset['target']
test_X = train_X
test_y = train_y

# add bias to the inputs
train_X = np.hstack([train_X, np.ones((train_X.shape[0], 1))])
test_X = np.hstack([test_X, np.ones((test_X.shape[0], 1))])

In [3]:
# wrap up with Variable in pytorch
train_X = Variable(torch.Tensor(train_X).float())
test_X = Variable(torch.Tensor(test_X).float())
train_y = Variable(torch.Tensor(train_y).long())
test_y = Variable(torch.Tensor(test_y).long())

In [4]:
shared_weight = Variable(torch.Tensor([1]))

model = Model(shared_weight, 5,2,2,3) # 5 ins (including one bias), 3 hidden layers with 2 nodes each, 3 outputs 

model.init_weights()

#train(model, 1000)
#evaluate(model)

In [5]:
criterion = torch.nn.CrossEntropyLoss()# cross entropy loss
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2)

def train(model, n_epochs=2000):
    
    for epoch in range(n_epochs):
        optimizer.zero_grad()

        batch = train_X.unsqueeze(dim=0)
        out = model(batch).view(-1, 3)
        loss = criterion(out, train_y)
        loss.backward()
        
        # set masked gradients to zero, so the weights will not be updated
        model.reset_frozen_grads()
        
        optimizer.step()

    print ('loss', loss.data)

def evaluate(model):
    predict_out = model(test_X.unsqueeze(dim=0)).view(-1, 3)
    _, predict_y = torch.max(predict_out, 1)

    print ('prediction accuracy', accuracy_score(test_y.data, predict_y.data))

#    print ('macro precision', precision_score(test_y.data, predict_y.data, average='macro'))
#    print ('micro precision', precision_score(test_y.data, predict_y.data, average='micro'))
#    print ('macro recall', recall_score(test_y.data, predict_y.data, average='macro'))
#    print ('micro recall', recall_score(test_y.data, predict_y.data, average='micro'))
    

In [6]:
epochs = 50

for i in range(epochs):
    model.freeze_weights(ratio=1/(epochs-i))
    train(model, 20)
    model.freeze_act_funcs(ratio=1/(epochs-i))
    train(model, 20)
    if i % 4 == 0:
        print (f"--- Iteration {i} ---")
        evaluate(model)

print("=== Final ===")
evaluate(model)

loss tensor(1.0945)
loss tensor(1.0918)
--- Iteration 0 ---
prediction accuracy 0.36666666666666664
loss tensor(1.0787)
loss tensor(1.0734)
loss tensor(1.0880)
loss tensor(1.0838)
loss tensor(1.0714)
loss tensor(1.0662)
loss tensor(1.0268)
loss tensor(1.0237)
--- Iteration 4 ---
prediction accuracy 0.66
loss tensor(0.9690)
loss tensor(0.9669)
loss tensor(1.1615)
loss tensor(1.0618)
loss tensor(0.9681)
loss tensor(0.9417)
loss tensor(1.1386)
loss tensor(1.0892)
--- Iteration 8 ---
prediction accuracy 0.35333333333333333
loss tensor(1.0309)
loss tensor(1.0040)
loss tensor(0.9259)
loss tensor(0.9230)
loss tensor(0.9341)
loss tensor(0.9282)
loss tensor(0.9173)
loss tensor(0.9095)
--- Iteration 12 ---
prediction accuracy 0.66
loss tensor(0.9083)
loss tensor(0.9053)
loss tensor(0.9041)
loss tensor(0.9034)
loss tensor(0.9030)
loss tensor(0.9035)
loss tensor(0.9031)
loss tensor(0.9028)
--- Iteration 16 ---
prediction accuracy 0.66
loss tensor(0.9026)
loss tensor(0.9024)
loss tensor(0.9022)
los

In [7]:
raise RuntimeWarning("Skipping the rest")

RuntimeWarning: Skipping the rest

In [ ]:
epochs = 20

for i in range(epochs):
    model.freeze_act_funcs(ratio=1/(epochs-i))
    
    train(model, 100)
    
    if i % 4 == 0:
        print (f"--- Iteration {i} ---")
        evaluate(model)
    
print ("=== Final ===")
model.freeze_act_funcs(ratio=1)
evaluate(model)

print("=== Longer Retrain ===")
train(model, 1000)
evaluate(model)

In [ ]:
epochs = 20

for i in range(epochs):
    model.freeze_weights(ratio=1/(epochs-i))
    train(model, 50)
    if i % 4 == 0:
        print (f"--- Iteration {i} ---")
        evaluate(model)

print("=== Final ===")
evaluate(model)

In [ ]:
model.network[2].linear.weight